In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from pathlib import Path
from sqlalchemy import create_engine
from ds100_utils import fetch_and_cache


plt.rcParams['figure.figsize'] = (4, 4)
plt.rcParams['figure.dpi'] = 150
sns.set()

In [ ]:
# From https://github.com/susam/heart/blob/master/heart.py
x = np.linspace(-1, 1, 10001)
y1 = np.sqrt(1 - np.abs(x)) * np.sqrt(np.abs(x))
y2 = (-3/2) * np.sqrt(1 - np.sqrt((np.abs(x))))
plt.plot(x, y1, color='red')
plt.plot(x, y2, color='red');

In [ ]:
import sqlite3
conn = sqlite3.connect('test.db')

In [ ]:
conn.executescript("""
DROP TABLE IF EXISTS s;
DROP TABLE IF EXISTS t;

CREATE TABLE s(
    m INTEGER PRIMARY KEY, 
    u TEXT
);

INSERT INTO s VALUES 
    (1, 'W'),
    (2, 'X'),
    (3, 'X'),
    (4, 'Y');

CREATE TABLE t(
    n TEXT PRIMARY KEY, 
    v CHAR
);

INSERT INTO t VALUES 
    ('A', 'X'),
    ('B', 'X'),
    ('C', 'Y'),
    ('D', 'Z');
""");

In [ ]:
def print_sql(s):
    print('>', s)
    for result in conn.execute(s):
        print(result)

print_sql('SELECT * FROM s;')
print_sql('SELECT * FROM t;')

In [ ]:
print_sql('SELECT * FROM s, t;')

In [ ]:
print_sql('SELECT * FROM s JOIN t ON s.u = t.v;')
print_sql('SELECT * FROM s, t WHERE s.u = t.v;')


In [ ]:
print_sql('SELECT * FROM t LEFT JOIN s ON s.u = t.v')

In [ ]:
print_sql('SELECT s.m, s.u, t.n, t.v FROM t LEFT JOIN s ON s.u = t.v;')
print_sql('SELECT s.m, s.u, t.n, t.v FROM s LEFT JOIN t ON s.u = t.v UNION '
          'SELECT s.m, s.u, t.n, t.v FROM t LEFT JOIN s ON s.u = t.v;')

In [ ]:
conn.close()  # DON'T forget to close the connection to the db

In [ ]:
# From https://www.imdb.com/interfaces/
fetch_and_cache('https://datasets.imdbws.com/title.basics.tsv.gz', 'titles.tsv.gz')
fetch_and_cache('https://datasets.imdbws.com/name.basics.tsv.gz', 'names.tsv.gz')
!gunzip -kf data/titles.tsv.gz
!gunzip -kf data/names.tsv.gz
!ls -lh data

In [ ]:
!head -n5 data/titles.tsv

In [ ]:
!head -n5 data/names.tsv

In [ ]:
!wc data/*.tsv

In [ ]:
# How to crash your kernel:
# pd.read_csv('data/names.tsv', sep='\t')

In [ ]:
# Instead, use sqlite3 (must be run from the terminal)
"""
$ sqlite3 imdb.db
sqlite> .mode tabs
sqlite> .import data/titles.tsv titles
sqlite> .import data/names.tsv names
""";

In [ ]:
conn = sqlite3.connect('imdb.db')
print_sql('SELECT * FROM titles LIMIT 10;')

In [ ]:
print_sql("SELECT name FROM sqlite_master WHERE type='table';")  # get all the tables in current database

In [ ]:
print_sql("PRAGMA table_info('titles');")  # get the schema of table 'titles'

In [ ]:
# Another way to get the schema
for exp in conn.execute('SELECT sql FROM sqlite_master;'):
    print(exp[0])

In [ ]:
# Python string substitution (new in Python 3.6)
who, where = 'John', 'here'
f"{who} is {where}"

In [ ]:
key = 'tconst'
title = 'primaryTitle'
time = 'runtimeMinutes'
year = 'startYear'
adult = 'isAdult'
kind = 'titleType'

select_romance = f'''
    SELECT {key} AS id,
           {title} AS title, 
           CAST({time} AS int) AS time,
           CAST({year} AS int) AS year
    FROM titles
    WHERE genres LIKE '%Romance%' AND
          year > 1900 AND 
          {adult} = '0' AND
          {kind} = 'movie' AND
          time > 60 AND time < 180
    '''
conn.executescript(f'''
    DROP TABLE IF EXISTS romance;
    CREATE TABLE romance AS {select_romance};
    ''')
print_sql('SELECT * FROM romance LIMIT 10')

In [ ]:
df = pd.read_sql('SELECT * FROM romance', conn)
df.head()

In [ ]:
df.shape

In [ ]:
df.hist('year');

In [ ]:
sns.scatterplot(df['year'], df['time']);

In [ ]:
sns.kdeplot(df['year'], df['time']);

In [ ]:
df['time'].groupby(df['year']).mean().plot();

In [ ]:
df['decade'] = (df['year'] // 10) * 10
sns.boxplot(x='decade', y='time', data=df)
plt.xticks(rotation=45);

In [ ]:
pd.read_sql('SELECT * FROM romance ORDER BY RANDOM() LIMIT 3', conn)

In [ ]:
three_years = '''
    SELECT year FROM romance 
         GROUP BY year 
         ORDER BY RANDOM() LIMIT 3
    '''
cluster_sample = f'''
    SELECT * FROM romance 
    WHERE year IN ({three_years})
    '''
pd.read_sql(f'''
    SELECT year, COUNT(*) 
    FROM ({cluster_sample}) 
    GROUP BY year
    ''', conn)

In [ ]:
pd.read_sql('SELECT * FROM names LIMIT 10;', conn)

In [ ]:
name = 'primaryName'
known = 'knownForTitles'
profession = 'primaryProfession'
born = 'birthYear'

select_actors = f'''
    SELECT {name} AS name, 
           CAST({born} AS int) AS born,
           SUBSTR({known}, 0, INSTR({known}, ',')) AS movie_id,
           CASE WHEN {profession} LIKE '%actor%' THEN 'actor' 
                WHEN {profession} LIKE '%actress%' THEN 'actress'   
                END AS profession
    FROM names
    WHERE {profession} LIKE '%act%' AND
          born > 1800
    '''
pd.read_sql(f'{select_actors} LIMIT 10', conn)

In [ ]:
select = f'''
    SELECT name AS name, profession, born, title, year FROM
    ({select_actors}) JOIN
    romance
    ON movie_id = id;
    '''
conn.executescript(f'''
    DROP TABLE IF EXISTS romantics;
    CREATE TABLE romantics AS {select};
    ''')
pd.read_sql('SELECT * FROM romantics LIMIT 10', conn)

In [ ]:
pd.read_sql('''
    SELECT name, born, year,
       CASE WHEN born < 1980 THEN 'old' 
            WHEN born < 2000 THEN 'not too old' 
            ELSE 'young' END AS age,
       CASE year % 10 WHEN 0 THEN 'start of decade'
                      WHEN 5 THEN 'middle of decade'
                      END AS time
    FROM romantics
    ''', conn).sample(10)

In [ ]:
df = pd.read_sql('SELECT * FROM romantics', conn)
df.shape

In [ ]:
df['age'] = df['year'] - df['born']
bins = np.linspace(0, 100, 21)
sns.distplot(df[df['profession']=='actor']['age'], bins=bins)
sns.distplot(df[df['profession']=='actress']['age'], bins=bins)
plt.legend(labels=['Actor', 'Actress'])
plt.xticks(bins, rotation=90)
plt.xlim(0, 100)
plt.ylabel('density');
plt.title('Ages of actors in the Romance movies they are known for');

In [ ]:
conn.close()